In [30]:
import networkx as nx
import random

In [31]:
G = nx.read_edgelist("dataset_ubicomp2013_tips_final_subcompontents_removed.txt")

Simple sampling, we can skip this part

In [32]:
k = 800
sampled_nodes = random.sample(G.nodes, k)
sampled_graph = G.subgraph(sampled_nodes)

C:\Users\MUSIC\AppData\Local\Temp/ipykernel_12328/232423041.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  sampled_nodes = random.sample(G.nodes, k)


Ther was a problem with a "freezed" graph, that can means kind a protection, so code unfreeze it to be able to remove small clusters

In [33]:
unfrozen_graph = nx.Graph(sampled_graph)
nx.is_frozen(unfrozen_graph)

False

Below needs after the sampling, since small clusters have in the graphs

In [34]:
max([len(c) for c in sorted(nx.connected_components(unfrozen_graph), key=len, reverse=True)])

109

In [35]:
compfix = [len(c) for c in sorted(nx.connected_components(unfrozen_graph), key=len, reverse=True)]
print("components in the dataset: ", len(compfix))
print([len(c) for c in sorted(nx.connected_components(unfrozen_graph), key=len, reverse=True)])
biggest_comp = max([len(c) for c in sorted(nx.connected_components(unfrozen_graph), key=len, reverse=True)])

components in the dataset:  586
[109, 14, 14, 6, 5, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

Remove clusters which are smaller than the biggest one

In [36]:
# remove the sub clusters

for component in list(nx.connected_components(unfrozen_graph)):
    if len(component)<biggest_comp:
        for node in component:
            unfrozen_graph.remove_node(node)

Double check

In [37]:
compfix = [len(c) for c in sorted(nx.connected_components(unfrozen_graph), key=len, reverse=True)]
print("components in the dataset: ", len(compfix))
print([len(c) for c in sorted(nx.connected_components(unfrozen_graph), key=len, reverse=True)])

components in the dataset:  1
[109]


Preferential Attachment method for the new connections

In [40]:
linkpred_pa = list(nx.preferential_attachment(unfrozen_graph))

linkpred_pa.sort(key = lambda tup: tup[2], reverse = True)


Here we just neglect the lines, where user connect to user or venue to venue.

In [41]:
recommends_list_pa = []
for i in range(len(linkpred_pa)):
    if (linkpred_pa[i][0][0] == "u" and linkpred_pa[i][1][0] == "v") or (linkpred_pa[i][0][0] == "v" and linkpred_pa[i][1][0] == "u"):
        recommends_list_pa.append(linkpred_pa[i])
    else:
        None
recommends_list_pa[:5]

[('v12621', 'u72', 88),
 ('u72', 'v7796', 88),
 ('v11056', 'u72', 66),
 ('v4556', 'u72', 66),
 ('u72', 'v19644', 66)]

# With the big dataset

In [42]:
G = nx.read_edgelist("dataset_ubicomp2013_tips_final_subcompontents_removed.txt")

In [43]:
linkpred_pa = list(nx.preferential_attachment(G))

linkpred_pa.sort(key = lambda tup: tup[2], reverse = True)

In [44]:
recommends_list_pa = []
for i in range(len(linkpred_pa)):
    if (linkpred_pa[i][0][0] == "u" and linkpred_pa[i][1][0] == "v") or (linkpred_pa[i][0][0] == "v" and linkpred_pa[i][1][0] == "u"):
        recommends_list_pa.append(linkpred_pa[i])
    else:
        None
recommends_list_pa[:20]

[('u82569', 'v11100', 4312),
 ('v12833', 'u82569', 3920),
 ('u82569', 'v564', 3920),
 ('v985', 'u82569', 3724),
 ('u82569', 'v17698', 3724),
 ('v654', 'u82569', 3528),
 ('u82569', 'v6293', 3528),
 ('v5744', 'u82569', 3332),
 ('u82569', 'v11954', 3332),
 ('v1211', 'u82569', 3136),
 ('u82569', 'v4156', 3136),
 ('u82569', 'v10965', 3136),
 ('u82569', 'v12700', 3136),
 ('v17415', 'u82569', 2940),
 ('v15071', 'u82569', 2940),
 ('u82569', 'v7913', 2940),
 ('u82569', 'v9250', 2940),
 ('u82569', 'v20', 2940),
 ('u82569', 'v4561', 2940),
 ('u82569', 'v23209', 2940)]